In [10]:
import nltk
import sqlite3
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd





In [11]:
print("Loading Data From DB")

targetDatabase = 'engA.db'
con = sqlite3.connect(targetDatabase)
db = pd.read_sql_query("""SELECT sid, cid, clemma, tag, tags FROM concept""",con)

print("Data Loaded!")



Loading Data From DB
Data Loaded!


In [ ]:
# DataSanitization

# TODO: Implement this based on the data
db.head(5)


In [12]:
# for each word, how many instance
#  for each word instance, list tags
#   for each tags, how many times the tag appear in the word

def tagsCount(row):
    tagsString = row['tags']
    if tagsString == None:
        return 0
    else:
        return len(row['tags'].split("; "))

db['tagsCount'] =  db.apply(tagsCount, axis=1)
# db.head(20)
print("Done!")

Done!


In [13]:
numWords = db.groupby('clemma').size()
wordsDB = db.drop(['sid', 'cid','tag'], axis = 1)
# wordsDB.head(5)
numWords = numWords.to_frame()
numWords.reset_index(level=0, inplace=True)
numWords.rename(columns={ numWords.columns[0]: "clemma" , numWords.columns[1]: 'wordCount'}, inplace = True)
numWords.head(5)
print("Done!")

Done!


In [14]:
allTags = db.drop(['clemma','sid', 'cid','tags','tagsCount'], axis = 1)
allTags = allTags.drop_duplicates()

allWords = db.drop(['tag','sid', 'cid','tags','tagsCount'], axis = 1)
allWords = allWords.drop_duplicates()

allWordCounts = db.groupby(['clemma']).size()
print("Done!")

Done!


In [15]:

wordsDB = pd.merge(wordsDB,numWords, on="clemma")
wordsDB = wordsDB.drop_duplicates()
# wordsDB.head(10)
def stringToList(string):
    if string == None:
        return []
    else:
        return string.split('; ')

wordsDB['tags'] = wordsDB.apply(lambda x : stringToList(x['tags']), axis =1  )
a= wordsDB.groupby(['clemma']).agg({'tags': "sum"})
a.reset_index(level=0, inplace=True)
a['tags'] = a.apply(lambda r: set(r['tags']), axis = 1)
a['tagsCount'] = a.apply(lambda r: len(r['tags']), axis = 1)
a = a.sort_values(['tagsCount'], ascending = False )

In [16]:
tagCount=0

def getTagCount(row):
    global tagCount
    tagCount+=1      
    word = row['clemma']
    print(word, tagCount*100/len(allWords), "percent Complete")
    tags = dict.fromkeys(row['tags'], 0)
    strippedTags = {}
    s = 0
    sumTags=0
    tags['w'] = 0
    tags['x'] = 0
    tags['pers'] = 0
    for tag in tags.keys():
        # TODO: CHANGED HERE
        strippedTag = tag.strip()
        if tag != strippedTag:
            print(f"'{tag}', '{strippedTag}'")
        count = len(db[ (db['clemma'] == word) & ( db['tag'] == strippedTag) ].index)
        strippedTags[strippedTag] = count
        sumTags+=count
        n = count/allWordCounts[word]
        if n == 0:
            continue
        nlogn = n * math.log10(n)
        s += abs(nlogn)

    if (s>1):
        print("ANOMALY!", word )
        print("word Count:",allWordCounts[word], "tag: ",sumTags )
    return (s,strippedTags)

b  = a.apply(getTagCount, axis =1)
# len(db[ (db['clemma'] == word) & ( db['tags'].str.contains(tag)) ])
# a['tagWithCounts'] = a.apply(getTagCount, axis =1)
#d=  a.head(10)
        

43930420531159 percent Complete
middle watch 98.44302901627742 percent Complete
murano 98.44675382724327 percent Complete
murtabak 98.4504786382091 percent Complete
medical instrument 98.45420344917495 percent Complete
no more 98.4579282601408 percent Complete
nine-to-five 98.46165307110664 percent Complete
nip off 98.46537788207249 percent Complete
nirvana 98.46910269303832 percent Complete
nite owl 98.47282750400417 percent Complete
no doubt 98.47655231497002 percent Complete
no end 98.48027712593586 percent Complete
no one 98.4840019369017 percent Complete
non-governmental 98.48772674786754 percent Complete
no signboard seafood 98.49145155883339 percent Complete
no way 98.49517636979924 percent Complete
nokia 98.49890118076507 percent Complete
nom de plume 98.50262599173092 percent Complete
non 98.50635080269676 percent Complete
non-authorised 98.51007561366261 percent Complete
nine times 98.51380042462846 percent Complete
nin'i 98.51752523559429 percent Complete
nike 98.52125004656

In [17]:
#MUST EXECUTE
a["results"] = b
pd.set_option('display.max_columns', 0)
pd.set_option('display.max_rows', 0)
pd.set_option('display.max_colwidth', -1)
a

,clemma,tags,tagsCount,results
3246,break,"{01972976-v, 07313814-n, 01597995-v, 00107739-v, 02625187-v, 01106864-v, 02681639-v, 01610666-v, 00334996-v, 00188007-n, 00482892-n, 01610463-v, 02681524-v, 00202784-v, 00390215-v, 02746449-v, 02318165-v, 80001019-v, 00030463-v, 02478469-v, 00258665-v, 15271008-n, 00383952-n, 00294748-n, 02399331-v, 14292090-n, 02029369-v, 00933821-v, 09278537-n, 00558536-v, 02073233-v, 01228524-v, 01369346-v, 07316856-n, 01785395-v, 07297633-n, 00738314-v, 02600082-v, 00500280-n, 00107604-v, 02559395-v, 00434374-v, 00155046-v, 00339464-v, 00738440-v, 02566528-v, 01989053-v, 01896484-v, 00778745-v, 01369758-v, 00364064-v, 00744439-v, 07367548-n, 02668523-v, 00334186-v, 01298931-v, 00336260-v, 01062997-n, 00551412-v, 02431320-v, 01557646-v, 01991347-v, 00935987-v, 02101373-v, 00202569-v, 00376400-n, 02535716-v, 00779061-v, 02570684-v, 01228396-v, 00059989-n, 02030764-v, 07367812-n, 00258366-v, 00362348-v, 02746617-v}",76,"(0.3597136321007663, {'01972976-v': 0, '07313814-n': 1, '01597995-v': 0, '00107739-v': 0, '02625187-v': 0, '01106864-v': 3, '02681639-v': 0, '01610666-v': 0, '00334996-v': 1, '00188007-n': 0, '00482892-n': 0, '01610463-v': 0, '02681524-v': 0, '00202784-v': 0, '00390215-v': 0, '02746449-v': 2, '02318165-v': 0, '80001019-v': 0, '00030463-v': 1, '02478469-v': 0, '00258665-v': 0, '15271008-n': 1, '00383952-n': 1, '00294748-n': 0, '02399331-v': 0, '14292090-n': 0, '02029369-v': 0, '00933821-v': 1, '09278537-n': 0, '00558536-v': 1, '02073233-v': 0, '01228524-v': 0, '01369346-v': 0, '07316856-n': 0, '01785395-v': 0, '07297633-n': 0, '00738314-v': 0, '02600082-v': 1, '00500280-n': 0, '00107604-v': 0, '02559395-v': 1, '00434374-v': 0, '00155046-v': 0, '00339464-v': 0, '00738440-v': 0, '02566528-v': 2, '01989053-v': 1, '01896484-v': 0, '00778745-v': 0, '01369758-v': 0, '00364064-v': 5, '00744439-v': 0, '07367548-n': 0, '02668523-v': 0, '00334186-v': 1, '01298931-v': 0, '00336260-v': 0, '01062997-n': 3, '00551412-v': 1, '02431320-v': 1, '01557646-v': 0, '01991347-v': 0, '00935987-v': 0, '02101373-v': 0, '00202569-v': 0, '00376400-n': 0, '02535716-v': 3, '00779061-v': 0, '02570684-v': 1, '01228396-v': 1, '00059989-n': 1, '02030764-v': 0, '07367812-n': 2, '00258366-v': 0, '00362348-v': 0, '02746617-v': 0, 'w': 0, 'x': 25, 'pers': 0})"
6052,cut,"{02137070-a, 14429885-n, 01225997-n, 01915730-v, 00292507-v, 01756416-v, unk, 00352331-n, 01754915-v, 01610990-v, 01754737-v, 00429060-v, 01510576-v, 00489475-n, 00386915-n, 02101649-v, 02033295-v, 01860107-v, 01611123-v, 05750948-n, 14287113-n, 00226071-v, 01666717-v, 03152743-n, 06721461-n, 01915865-v, 00387657-n, 13289159-n, 00067254-n, 06618234-n, 00473322-v, 06428216-n, 01756277-v, 07653394-n, 01566027-a, 00571444-n, 01319738-v, 01555437-v, 02613860-v, 00359238-n, 02134232-v, 00661278-a, 01227088-v, 00243900-v, 00756327-a, 01756563-v, 01859995-v, 01713491-v, 00447654-v, 00061219-v, 02454750-a, 01064799-v, 00882742-a, 00663267-a, 03152619-n, 01059564-v, 00201722-v, 01320424-v, 00429968-v, 00004615-a, 00561090-v, 00096136-v, 00565809-n, 01552519-v, 01413917-v, 02422283-v, 02588280-v, 06612865-n, 00108475-v, 00662958-a, 01610834-v}",71,"(0.4082514828490045, {'02137070-a': 0, '14429885-n': 0, '01225997-n': 0, '01915730-v': 1, '00292507-v': 1, '01756416-v': 0, 'unk': 0, '00352331-n': 0, '01754915-v': 0, '01610990-v': 0, '01754737-v': 1, '00429060-v': 8, '01510576-v': 0, '00489475-n': 0, '00386915-n': 1, '02101649-v': 0, '02033295-v': 0, '01860107-v': 0, '01611123-v': 0, '05750948-n': 1, '14287113-n': 0, '00226071-v': 0, '01666717-v': 0, '03152743-n': 0, '06721461-n': 0, '01915865-v': 0, '00387657-n': 1, '13289159-n': 0, '00067254-n': 0, '06618234-n': 0, '00473322-v': 0, '06428216-n': 0, '01756277-v': 0, '07653394-n': 0, '01566027-a': 0, '00571444-n': 0, '01319738-v': 0, '01555437-v': 0, '02613860-v': 0, '00359238-n': 0, '02134232-v': 1, '00661278-a': 2, '01227088-v': 1, '00243900-v': 0, '00756327-a': 0, '01756563-v': 0, '01859995-v': 0, '01713491-v': 0, '00

In [ ]:
pd.set_option('display.max_columns', 0)
pd.set_option('display.max_rows', 0)
pd.set_option('display.max_colwidth', -1)
a[ (a['clemma'] == "break" )]

In [18]:
import os
print(os.getcwd())
p = os.path.join(os.getcwd(), "PicklepeePumperrump.pckl")
print(p)
a.to_pickle(p)

c:\Users\iambe\Desktop\Eyes on the Inside\2020 - 2021 Semester 1\HG2051 - Language and the Computer\dev environ\dbs
c:\Users\iambe\Desktop\Eyes on the Inside\2020 - 2021 Semester 1\HG2051 - Language and the Computer\dev environ\dbs\PicklepeePumperrump.pckl


In [ ]:
print("Unpickling!")

rdb = pd.read_pickle('201601_engA_processed.pckl')
print("Unpickled!")
pd.set_option('display.max_rows', 0)
pd.set_option('display.max_columns', 0)
pd.set_option('display.max_colwidth', 0)
rdb

In [ ]:
rdb['entropyVal'] =  rdb.apply(lambda x: x['results'][0], axis =1)
rdb = rdb.sort_values(['entropyVal'],axis = 0, ascending= False )
rdb